# Taking advantage of Colab Pro



## Faster GPUs

With Colab Pro you have priority access to our fastest GPUs. For example, you may get a T4 or P100 GPU at times when most users of standard Colab receive a slower K80 GPU. You can see what GPU you've been assigned at any time by executing the following cell.

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sat Jun  6 17:56:29 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In order to use a GPU with your notebook, select the Runtime > Change runtime type menu, and then set the hardware accelerator dropdown to GPU.

## More memory

With Colab Pro you have the option to access high-memory VMs when they are available. To set your notebook preference to use a high-memory runtime, select the Runtime > 'Change runtime type' menu, and then select High-RAM in the Runtime shape dropdown.

You can see how much memory you have available at any time by running the following code.


In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Longer runtimes

All Colab runtimes are reset after some period of time (which is faster if the runtime isn't executing code). While Colab Pro subscribers still have limits, these will be roughly twice the limits for non-subscribers.

## Resource limits in Colab Pro

Your resources are not unlimited in Colab Pro. To make the most of Colab Pro, please avoid using resources when you don't need them. For example, only use a GPU or high-RAM runtime when required, and close Colab tabs when finished.


## Send us feedback!

If you have any feedback for us, please let us know. The best way to send feedback is by using the Help > 'Send feedback...' menu. If you encounter usage limits in Colab Pro and would be interested in a product with higher usage limits, do let us know.

If you encounter errors or other issues with billing (payments) for Colab Pro, please email colab-billing@google.com.

In [5]:
import os
import random
import numpy as np
import pandas as pd 
from skimage import io
from skimage import color
from PIL import Image
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
from dask.array.image import imread
from dask import bag, threaded
from dask.diagnostics import ProgressBar
import cv2
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")



import keras
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense,GlobalAveragePooling2D
from keras.layers import Flatten,Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image 
from keras.layers.normalization import BatchNormalization
from keras import optimizers


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [6]:
import pickle
train_image = []
image_label = []

for i in range(10):
  #path = "/content/drive/My Drive/kaggle/cache/train_r_224_c_224_c_3_class" + str(i) + ".dat"
  #path = "/content/drive/My Drive/distdrv/cache/mask_r_224_c_224_c_3_class" + str(i) + ".dat"
  path = "/content/drive/My Drive/distdrv/cache/zoommask_r_224_c_224_c_3_class" + str(i) + ".dat"


  print(f'loading pickle files from class = {i}')
  # get orig image
  file = open(path, 'rb')
  images, labels = pickle.load(file)
  train_image = train_image + images
  #image_label = image_label + labels





loading pickle files from class = 0
loading pickle files from class = 1
loading pickle files from class = 2
loading pickle files from class = 3
loading pickle files from class = 4
loading pickle files from class = 5
loading pickle files from class = 6
loading pickle files from class = 7
loading pickle files from class = 8
loading pickle files from class = 9


In [0]:
images = []
labels = []
driver_details = []

In [8]:
print(f'train image size = {len(train_image)}, label size = {len(labels)}')

train image size = 22424, label size = 0


In [0]:
import random

random.shuffle(train_image)



In [10]:
print(f'train image size = {len(train_image)}')

train image size = 22424


In [0]:
## getting list of driver names

D = []
for features,labels,drivers in train_image:
    D.append(drivers)

## Deduplicating drivers

deduped = []

for i in D:
    if i not in deduped:
        deduped.append(i)
    

## selecting random drivers for the validation set
driv_selected = []
import random
driv_nums = random.sample(range(len(deduped)), 4)
for i in driv_nums:
    driv_selected.append(deduped[i])


In [12]:
driv_nums

[0, 13, 21, 5]

In [13]:
len(deduped)

26

In [14]:
## Splitting the train and test

X_train= []
y_train = []
X_test = []
y_test = []
D_train = []
D_test = []

#for features,labels,drivers,features2,labels2,drivers2 in zip(train_image, train_image2):
for features, labels, drivers in train_image:

    if drivers in driv_selected:
        X_test.append(features)
        y_test.append(labels)
        #D_test.append(drivers)
    
    else:
        X_train.append(features)
        y_train.append(labels)
        #D_train.append(drivers)


true_test = y_test
    
print (len(X_train),len(X_test))
print (len(y_train),len(y_test))






19019 3405
19019 3405


In [15]:
len(X_train)

19019

In [16]:
print(y_train[0])

2


In [17]:
## Converting images to nparray. Encoding the Y


X_train = np.array(X_train).reshape(-1,224,224,3)
X_test = np.array(X_test).reshape(-1,224,224,3)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print (X_train.shape)
print (X_test.shape)
print (y_train.shape)
print (y_test.shape)


(19019, 224, 224, 3)
(3405, 224, 224, 3)
(19019, 10)
(3405, 10)


In [0]:
from __future__ import print_function  # for Python2
import sys

local_vars = list(locals().items())
for var, obj in local_vars:
  #print(var, sys.getsizeof(obj))
  pass

In [19]:
## Defining the input
## Defining the input

from keras.layers import Input
resnet50_input = Input(shape = (224, 224, 3), name = 'Image_input')

## The RESNET model

from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.applications.resnet50 import ResNet50


#Get the RESNET weights and layers

model_resnet50_conv = ResNet50(weights= 'imagenet', include_top=False, input_shape= (224,224,3))
model_resnet50_conv.summary()



94658560/94653016 [==============================] - 3s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________

In [20]:
#Use the generated model 
from keras.models import Model


output_resnet50_conv = model_resnet50_conv(resnet50_input)

#Add the fully-connected layers 

x=GlobalAveragePooling2D()(output_resnet50_conv)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x = Dropout(0.1)(x) # **reduce dropout 
x=Dense(1024,activation='relu')(x) #dense layer 2
x = BatchNormalization()(x)
x = Dropout(0.35)(x)
x=Dense(512,activation='relu')(x) #dense layer 3
x = Dense(10, activation='softmax', name='predictions')(x)


resnet50_pretrained = Model(input = resnet50_input, output = x)
# for layer in resnet50_pretrained.layers[:2]:
#     layer.trainable=False
# for layer in resnet50_pretrained.layers[2:]:
#     layer.trainable=True


resnet50_pretrained.summary()

# Compile CNN model
adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0)
sgd = optimizers.SGD(lr = 0.001)
resnet50_pretrained.compile(loss='categorical_crossentropy',optimizer = sgd,metrics=['accuracy'])


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Image_input (InputLayer)     (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096

In [21]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint,EarlyStopping

checkpointer = ModelCheckpoint('/content/drive/My Drive/kaggle/resnetmaskModel_aug.hdf5', verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='accuracy', patience=7, verbose=1)


datagen = ImageDataGenerator(
    height_shift_range=0.5,
    width_shift_range = 0.5,
    zoom_range = 0.5,
    rotation_range=30
        )
#datagen.fit(X_train)
data_generator = datagen.flow(X_train, y_train, batch_size = 64)

# Fits the model on batches with real-time data augmentation:
resnet50_model = resnet50_pretrained.fit_generator(data_generator,steps_per_epoch = len(X_train) / 64, callbacks=[checkpointer, earlystopper],
                                                            epochs = 40, verbose = 1, validation_data = (X_test, y_test))


Epoch 1/40
298/297 [==============================] - 241s 809ms/step - loss: 2.5895 - accuracy: 0.1501 - val_loss: 1.9393 - val_accuracy: 0.3254

Epoch 00001: val_loss improved from inf to 1.93935, saving model to /content/drive/My Drive/kaggle/resnetmaskModel_aug.hdf5
Epoch 2/40
298/297 [==============================] - 223s 748ms/step - loss: 2.0278 - accuracy: 0.3085 - val_loss: 1.2694 - val_accuracy: 0.5568

Epoch 00002: val_loss improved from 1.93935 to 1.26939, saving model to /content/drive/My Drive/kaggle/resnetmaskModel_aug.hdf5
Epoch 3/40
298/297 [==============================] - 227s 761ms/step - loss: 1.5450 - accuracy: 0.4738 - val_loss: 0.7806 - val_accuracy: 0.7374

Epoch 00003: val_loss improved from 1.26939 to 0.78056, saving model to /content/drive/My Drive/kaggle/resnetmaskModel_aug.hdf5
Epoch 4/40
298/297 [==============================] - 229s 770ms/step - loss: 1.2424 - accuracy: 0.5762 - val_loss: 0.6636 - val_accuracy: 0.7900

Epoch 00004: val_loss improved f

In [0]:
#vgg16_model.save("/content/drive/My Drive/kaggle/singleModel_aug.h5")
resnet50_pretrained.save_weights("/content/drive/My Drive/kaggle/resnetmaskModel_aug_weights.h5")

In [23]:
from keras.models import load_model
rcModel = load_model('/content/drive/My Drive/kaggle/resnetmaskModel_aug.hdf5')
rcModel.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Image_input (InputLayer)     (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096

In [24]:
X_test.shape

(3405, 224, 224, 3)

In [25]:
# labels is the image array

from sklearn.metrics import accuracy_score, confusion_matrix

model1_prediction = []
model1_pred_class = []

model1_prediction = rcModel.predict(X_test)
print('Images Predicted until now:',len(model1_prediction))
print(f'True images: {len(true_test)}')

for i in range(len(model1_prediction)):
    model1_pred_class.append(np.where(model1_prediction[i] == np.amax(model1_prediction[i]))[0][0])

    
print('The accuracy of this model over validation set is:',accuracy_score(true_test,model1_pred_class))
confusion_matrix(true_test,model1_pred_class)

Images Predicted until now: 3405
True images: 3405
The accuracy of this model over validation set is: 0.892511013215859


array([[257,   2,   0,   0,   1,   2,   1,   0,   4,  84],
       [  0, 310,   0,   0,   1,   0,   6,   0,  43,   1],
       [  1,   1, 351,   0,   0,   0,   0,   4,   5,   0],
       [  0,   0,   0, 346,   4,   2,   0,   0,   0,   0],
       [  0,   0,   0,   0, 357,   0,   1,   0,   3,   0],
       [  3,   0,   0,   0,   0, 353,   2,   0,   1,   0],
       [  0,   1,   0,   0,   0,   0, 324,   2,  12,   0],
       [  0,   0,   0,   0,   0,   0,   0, 299,   3,   0],
       [ 14,   0,   3,   0,  30,   0,   9,  12, 202,  21],
       [ 75,   0,   0,   0,   0,   2,   1,   9,   0, 240]])

In [26]:
print(f'prediction={model1_prediction}, shape={model1_prediction.shape}')

prediction=[[3.9262008e-02 2.6558555e-04 6.1509095e-04 ... 5.3974806e-04
  3.7960537e-02 1.2262423e-02]
 [1.8784506e-03 6.2892759e-05 1.8902203e-04 ... 5.0987706e-05
  5.0359398e-02 5.5679688e-03]
 [3.5427579e-06 8.9151598e-08 6.7137370e-07 ... 5.6298546e-07
  7.0244452e-05 9.6647345e-06]
 ...
 [7.9739628e-05 9.9951553e-01 3.0914666e-06 ... 1.6685738e-06
  1.9236861e-04 4.9351231e-05]
 [3.0475596e-06 4.1153478e-05 2.4769661e-01 ... 1.6736552e-03
  7.4237090e-01 2.0289193e-05]
 [4.3013215e-09 1.6257990e-12 5.8542440e-13 ... 1.4489042e-11
  1.0822424e-10 7.2565391e-11]], shape=(3405, 10)


In [27]:
from sklearn.metrics import log_loss

lgloss = log_loss(y_test, model1_prediction, eps=1e-15, normalize=True, sample_weight=None, labels=None)

print('The log loss from this model is:',round(lgloss,2))

The log loss from this model is: 0.39
